# Pilas (Stacks) y Colas (Queues) en Python – versión extendida
*Generado el 2025-06-02*

Este notebook cubre:
- Fundamentos teóricos (LIFO/FIFO)
- Implementaciones y variantes en Python
- Visualizaciones, casos de uso y eficiencia
- Ejemplos prácticos y avanzados
- Benchmark y buenas prácticas

## ¿Por qué estudiar pilas y colas?
Pilas y colas son la base de algoritmos como *undo/redo*, recorrido de grafos, procesamiento de tareas y más. Comprenderlas te permite resolver problemas reales de manera eficiente e idiomática en Python.

## 1. Conceptos fundamentales: LIFO y FIFO
- **LIFO (Last-In, First-Out)**: el último elemento en entrar es el primero en salir. Ejemplo: pila de platos.
- **FIFO (First-In, First-Out)**: el primer elemento en entrar es el primero en salir. Ejemplo: cola en una taquilla.
Ambas estructuras restringen el acceso para garantizar un orden concreto de procesamiento.

## 2. Stacks (Pilas)
### 2.1 ¿Qué es una pila?
Una pila es una estructura LIFO: solo puedes agregar (push) o quitar (pop) elementos por un único extremo (el 'tope').

### 2.2 Operaciones típicas
- **Push:** añade elemento al tope.
- **Pop:** extrae el elemento del tope.
- **Peek:** consulta el tope sin quitarlo.
- **is_empty:** ¿la pila está vacía?

### 2.3 Implementación básica con `list`
`list.append` y `list.pop()` actúan como push/pop. Las listas de Python son contiguas y eficientes para pilas.

In [1]:
class StackList:
    """Stack LIFO implementado con list"""
    def __init__(self): self._data = []
    def push(self, x):  self._data.append(x)
    def pop(self):      return self._data.pop()
    def peek(self):     return self._data[-1] if self._data else None
    def is_empty(self): return not self._data
    def __repr__(self): return f"StackList({self._data})"

s = StackList()
for val in [3, 5, 7]:
    s.push(val)
print('Pila tras pushes:', s)
print('Pop:', s.pop())
print('Peek:', s.peek())
print('Vacía?', s.is_empty())


Pila tras pushes: StackList([3, 5, 7])
Pop: 7
Peek: 5
Vacía? False


### 2.4 Implementación con `collections.deque`
`deque` es eficiente para operaciones en ambos extremos, pero para pila suele bastar con `list`.

In [2]:
from collections import deque
class StackDeque:
    """Stack LIFO implementado con deque"""
    def __init__(self): self._data = deque()
    def push(self, x):  self._data.append(x)
    def pop(self):      return self._data.pop()
    def peek(self):     return self._data[-1] if self._data else None
    def is_empty(self): return not self._data
    def __repr__(self): return f"StackDeque({list(self._data)})"

sd = StackDeque()
for v in 'XYZ':
    sd.push(v)
print('StackDeque:', sd)


StackDeque: StackDeque(['X', 'Y', 'Z'])


### 2.5 Visualización conceptual (ASCII)
```text
|   X   | <- tope
|   Y   |
|   Z   |
---------
```

### 2.6 Aplicaciones reales de pilas
- **Undo/Redo** en editores
- **Backtracking** (solucionadores, recorrido en profundidad)
- **Pila de llamadas** en recursión
- **Evaluación de expresiones** y parsing


## 3. Queues (Colas)
### 3.1 ¿Qué es una cola?
Una cola es una estructura FIFO: se inserta por un extremo (final) y se elimina por el otro (frente).

### 3.2 Operaciones típicas
- **Enqueue:** añade elemento al final
- **Dequeue:** extrae elemento del frente
- **Front:** consulta el elemento al frente
- **is_empty:** ¿cola vacía?

### 3.3 Implementación con `list` (no recomendado para dequeue)
`list.pop(0)` es O(n): desplaza todos los elementos. Solo usar en colas muy pequeñas.

In [3]:
class QueueList:
    """Cola FIFO usando list (sólo recomendada para pruebas o colas cortas)"""
    def __init__(self): self._data = []
    def enqueue(self, x): self._data.append(x)
    def dequeue(self):    return self._data.pop(0)
    def front(self):      return self._data[0] if self._data else None
    def is_empty(self):   return not self._data
    def __repr__(self):   return f"QueueList({self._data})"

ql = QueueList()
for n in [1, 2, 3]:
    ql.enqueue(n)
print('QueueList:', ql)
print('Dequeue:', ql.dequeue())
print('Front:', ql.front())


QueueList: QueueList([1, 2, 3])
Dequeue: 1
Front: 2


### 3.4 Implementación eficiente con `collections.deque`
`deque` es ideal para colas: `append` (enqueue) y `popleft` (dequeue) son O(1).

In [4]:
from collections import deque
class QueueDeque:
    """Cola FIFO usando deque"""
    def __init__(self): self._data = deque()
    def enqueue(self, x): self._data.append(x)
    def dequeue(self):    return self._data.popleft()
    def front(self):      return self._data[0] if self._data else None
    def is_empty(self):   return not self._data
    def __repr__(self):   return f"QueueDeque({list(self._data)})"

qd = QueueDeque()
for c in 'ABC':
    qd.enqueue(c)
print('QueueDeque:', qd)
print('Dequeue:', qd.dequeue())
print('Front:', qd.front())


QueueDeque: QueueDeque(['A', 'B', 'C'])
Dequeue: A
Front: B


### 3.5 Implementación concurrente con `queue.Queue`
`queue.Queue` permite colas seguras entre hilos: locking y wait integrados.

In [5]:
import queue, threading, time

def productor(q, n=3):
    for i in range(n):
        q.put(f'tarea-{i}')
        print(f'Productor: añade tarea-{i}')
        time.sleep(0.05)
    q.put(None)  # señal de parada

def consumidor(q):
    while True:
        t = q.get()
        if t is None: break
        print(f'Consumidor: procesa {t}')
        q.task_done()

q = queue.Queue()
h1 = threading.Thread(target=productor, args=(q,5), daemon=True)
h2 = threading.Thread(target=consumidor, args=(q,), daemon=True)
h1.start(); h2.start()
h1.join(); h2.join()


Productor: añade tarea-0
Consumidor: procesa tarea-0
Productor: añade tarea-1
Consumidor: procesa tarea-1
Productor: añade tarea-2
Consumidor: procesa tarea-2
Productor: añade tarea-3
Consumidor: procesa tarea-3
Productor: añade tarea-4
Consumidor: procesa tarea-4


### 3.6 Visualización conceptual (ASCII)
```text
[FRENTE] -> | 1 | 2 | 3 | <- [FINAL]
```

### 3.7 Aplicaciones reales de colas
- **Scheduling** y asignación de recursos
- **Recorridos BFS** en grafos
- **Spool de impresión y pipelines de datos**
- **Simulación de procesos y sistemas de atención


## 4. Comparativa de eficiencia y buenas prácticas
| Estructura          | Encolar/Push | Desencolar/Pop | Consulta tope/frente | Ideal para        |
|---------------------|--------------|----------------|----------------------|-------------------|
| `list` (pila)       | O(1)         | O(1)           | O(1)                 | Pilas (LIFO)      |
| `list` (cola)       | O(1)         | O(n)           | O(1)                 | Pequeñas colas    |
| `deque`             | O(1)         | O(1)           | O(1)                 | Pilas y colas     |
| `queue.Queue`       | O(1)         | O(1)           | O(1)                 | Multi-hilo        |

**Regla práctica:**
- Usa `list` sólo para pilas puras.
- Para colas, usa `deque` o `queue.Queue` si hay hilos.


## 5. Ejemplos avanzados

### 5.1 Undo/Redo con dos pilas

In [6]:
class UndoRedo:
    def __init__(self):
        self._done = []
        self._undone = []

    def do(self, act):
        self._done.append(act)
        self._undone.clear()

    def undo(self):
        if self._done:
            x = self._done.pop()
            self._undone.append(x)
            return x

    def redo(self):
        if self._undone:
            x = self._undone.pop()
            self._done.append(x)
            return x

ur = UndoRedo()
for action in "123":
    ur.do(f"accion-{action}")
print("Estado tras acciones:", ur._done)
print("Undo:", ur.undo())
print("Redo:", ur.redo())


Estado tras acciones: ['accion-1', 'accion-2', 'accion-3']
Undo: accion-3
Redo: accion-3


### 5.2 Simulación de impresora (cola FIFO)

In [8]:
from collections import deque
class Impresora:
    def __init__(self):
        self.cola = deque()
    def enviar_trabajo(self, doc):
        self.cola.append(doc)
        print(f"Enviado: {doc}")
    def imprimir(self):
        if self.cola:
            doc = self.cola.popleft()
            print(f"Imprimiendo: {doc}")
        else:
            print("Cola vacía.")

imp = Impresora()
for d in ['A.pdf', 'B.pdf', 'C.pdf']:
    imp.enviar_trabajo(d)
while imp.cola:
    imp.imprimir()


Enviado: A.pdf
Enviado: B.pdf
Enviado: C.pdf
Imprimiendo: A.pdf
Imprimiendo: B.pdf
Imprimiendo: C.pdf


## 5.3 Búsqueda en anchura (BFS) con `deque`


Este código implementa el algoritmo **BFS (Breadth-First Search)**, o **búsqueda en anchura**, sobre un grafo dirigido representado como diccionario de listas.  
- BFS recorre los nodos **nivel por nivel** desde un nodo inicial.
- Usa una **cola (FIFO)** para explorar primero los vecinos más cercanos.
- Es útil para encontrar caminos más cortos, realizar análisis por capas, resolver problemas en redes, etc.


In [7]:
from collections import deque
def bfs(grafo, inicio):
    visitados = {inicio}
    cola = deque([inicio])
    orden = []
    while cola:
        v = cola.popleft()
        orden.append(v)
        for w in grafo[v]:
            if w not in visitados:
                visitados.add(w)
                cola.append(w)
    return orden

G = {'A': ['B','C'], 'B':['D'], 'C':['D'], 'D':[]}
print("Recorrido BFS:", bfs(G, 'A'))


Recorrido BFS: ['A', 'B', 'C', 'D']


BFS explora los nodos más cercanos primero:

A → explora B y C.

Luego visita los vecinos de B y C (que es D en ambos casos).

Esto permite recorrer por niveles.

#### ¿Por qué se usa una cola?
Porque queremos visitar los nodos en el orden en que fueron descubiertos.

El primero que entra es el primero que se explora.

Esto garantiza que se visita cada nodo en la menor cantidad de pasos desde el inicio.

#### Aplicaciones típicas de BFS
Encontrar el camino más corto en grafos no ponderados.

Recorrer árboles por niveles (level order traversal).

Detección de ciclos en grafos no dirigidos.

Solución de problemas de redes (ruteo, propagación).

Generación de soluciones por capas (búsqueda exhaustiva).

### 5.4 Backtracking básico usando pila (DFS en laberinto)

Simula el **recorrido de un laberinto** (grafo) usando una **pila explícita** para realizar un *Depth-First Search* (DFS, búsqueda en profundidad) de forma iterativa.  
- **DFS** explora siempre el último camino abierto antes de retroceder (“backtracking”).
- El uso de una **pila** permite recordar por dónde hemos venido y retroceder si es necesario.
- Este patrón es muy útil en laberintos, juegos de puzzles, árboles de decisión, etc.

---

In [11]:
maze = {
    'A': ['B', 'C'],
    'B': ['D'],
    'C': [],
    'D': []
}
def dfs(maze, start):
    stack, visitado = [start], set()
    orden = []
    while stack:
        v = stack.pop()
        if v not in visitado:
            orden.append(v)
            visitado.add(v)
            stack.extend(maze[v][::-1])
    return orden

print("DFS (pila explícita):", dfs(maze, 'A'))


DFS (pila explícita): ['A', 'B', 'D', 'C']


DFS visitará en orden: A, B, D, C

### Explicación intuitiva
Imagina recorrer un laberinto:

- Cada vez que llegas a una bifurcación, eliges un camino (el último que encontraste).

- Si llegas a un callejón sin salida, retrocedes al último cruce pendiente (la pila recuerda esto).

- Así, exploras todos los caminos posibles, uno a uno.

¿Por qué usar una pila?
Una función recursiva también usa una “pila de llamadas” interna.

Usar una pila explícita permite controlar el proceso y evitar errores por recursión profunda.

Diferencia con BFS
- DFS (pila): explora en profundidad, va hasta el final de un camino antes de probar otro.
- BFS (cola): explora en anchura, primero recorre todos los nodos de un nivel antes de bajar al siguiente.




## 6. Micro-benchmark: `list` vs `deque` para dequeue
Probamos desencolar 30.000 elementos de la izquierda.

In [10]:
import timeit
setup = """
from collections import deque
N = 30000
lst = list(range(N))
dq = deque(range(N))
"""
print('list.pop(0):', timeit.timeit('while lst: lst.pop(0)', setup=setup, number=1))
print('deque.popleft():', timeit.timeit('while dq: dq.popleft()', setup=setup, number=1))


list.pop(0): 0.06149532699987503
deque.popleft(): 0.0012254099999609025


## 7. Resumen y mejores prácticas
- **Stack:** usa `list` para LIFO puro, `deque` si quieres doble extremo.
- **Queue:** usa `deque` para FIFO, `queue.Queue` para hilos.
- **Evita** `list.pop(0)` para colas largas: es O(n).
- **Acumula siempre los datos antes de usarlos si la operación dominante lo requiere.
- **Visualiza** el flujo con diagramas simples para comprender el orden de procesamiento.
